In [1]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import json

import evaluate
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    AutoModelForCausalLM
)
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# Seed Set
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# 디바이스 설정 (GPU가 사용 가능하면 GPU를 사용하고, 그렇지 않으면 CPU 사용)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [2]:

model_id = "sh2orc/Llama-3.1-Korean-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
# Few shot
with open("./prompt/few_shot_ko.json") as f:
    few_shot = json.load(f)
few_shot_list = [few_shot[d] for d in few_shot]
few_shot_list

[{'input': '러시아 시리아서 S500 발사했나…러 국방부는 부인',
  'output': '러시아 시리아서 S500 발사했나…러 국방부는 부인'},
 {'input': '120개 언어 인도 힌디어 국어 지정 놓고 찬반 논란',
  'output': '120개 언어 인도 힌디어 국어 지정 놓고 찬반 논란'},
 {'input': '무선 장비로 침입자 막는다… KT텔레캅 새 보안 서비스종합',
  'output': '무선 장비로 침입자 막는다… KT텔레캅 새 보안 서비스종합'},
 {'input': '프로농구 인삼공사 컬페퍼 햄스트링 부상…교체할 듯',
  'output': '프로농구 인삼공사 컬페퍼 햄스트링 부상…교체할 듯'},
 {'input': 'LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택',
  'output': 'LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택'},
 {'input': '서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로',
  'output': '서울문고, 영풍그룹에 인수…오프라인서점 교보-영풍 양강구도로'},
 {'input': '프로야구~롯TKIAs광주 경기 y천취소', 'output': '프로야구 롯데-KIA 광주 경기 우천취소'},
 {'input': '발언하\\ 박광E', 'output': '발언하는 박광온'},
 {'input': 'xW리 a)엔 예비후0V사전여론조사 결과 유출 논c',
  'output': '새누리, 이번엔 예비후보 사전여론조사 결과 유출 논란'},
 {'input': '美 中g위구르`탄압cN공…중국은 종w와 R쟁 중',
  'output': "美, 中 '위구르 탄압' 맹공…'중국은 종교와 전쟁 중'"},
 {'input': '조국U압수수색f검2와 통7 파장…한h·바(미래 &핵K진i',
  'output': "조국 '압수수색 검사와 통화' 파장…한국·바른미래 '탄핵추진'"},
 {'input': "^r^[P홀w2019년H'v요?서트vYp…세,#uNs[cJ ", '

In [ ]:
def replace_ascii_with_mask(text):
    # 결과를 저장할 리스트
    result = []
    
    # 입력 문자열을 한 문자씩 순회
    for char in text:
        # 공백은 그대로 추가
        if char == ' ':
            result.append(char)
        # ASCII 코드 범위(0 ~ 127) 확인, 공백 제외
        elif ord(char) <= 127:
            # ASCII 문자면 <mask>로 대체
            result.append('_')
        else:
            # 그렇지 않으면 원래 문자를 그대로 추가
            result.append(char)
    
    # 리스트를 문자열로 변환하여 반환
    return ''.join(result)

In [9]:
input_text = "산업생산jQ8째 L고 제조1가동률은-금융위jS수yk합2보"
replace_text = replace_ascii_with_mask(input_text)

system_message = """당신은 뉴스 기사 제목을 복원하는 어시스턴트로, 한 문장 내외의 뉴스 기사 제목을 복원해야합니다. 다음 지침을 따르세요.

1. 주어진 제목이 훼손되지 않았다면, 수정하지마세요.
2. 주어진 제목에서 _가 존재한다면 해당 부분을 의미가 이어지도록 다른 문장으로 바꿔주세요.
3. 제목이 너무 손상되어 복구가 불가능하다면, "복구 불가"라고 답하세요.
"""
few_shot_messages = []
for few_shot_data in few_shot_list:
    few_shot_messages.append(
        {"role":"user","content":few_shot_data["input"]}
    )
    few_shot_messages.append(
        {"role":"assistant","content":few_shot_data["output"]}
    )
messages = [
    [{"role":"system", "content":system_message}] +
    few_shot_messages +
    [{"role":"user", "content":replace_text}]
]
print(messages)

[[{'role': 'system', 'content': '당신은 뉴스 기사 제목을 복원하는 어시스턴트로, 한 문장 내외의 뉴스 기사 제목을 복원해야합니다. 다음 지침을 따르세요.\n\n1. 주어진 제목이 훼손되지 않았다면, 수정하지마세요.\n2. 주어진 제목에서 _가 존재한다면 해당 부분을 의미가 이어지도록 다른 문장으로 바꿔주세요.\n3. 제목이 너무 손상되어 복구가 불가능하다면, "복구 불가"라고 답하세요.\n'}, {'role': 'user', 'content': '러시아 시리아서 S500 발사했나…러 국방부는 부인'}, {'role': 'assistant', 'content': '러시아 시리아서 S500 발사했나…러 국방부는 부인'}, {'role': 'user', 'content': '120개 언어 인도 힌디어 국어 지정 놓고 찬반 논란'}, {'role': 'assistant', 'content': '120개 언어 인도 힌디어 국어 지정 놓고 찬반 논란'}, {'role': 'user', 'content': '무선 장비로 침입자 막는다… KT텔레캅 새 보안 서비스종합'}, {'role': 'assistant', 'content': '무선 장비로 침입자 막는다… KT텔레캅 새 보안 서비스종합'}, {'role': 'user', 'content': '프로농구 인삼공사 컬페퍼 햄스트링 부상…교체할 듯'}, {'role': 'assistant', 'content': '프로농구 인삼공사 컬페퍼 햄스트링 부상…교체할 듯'}, {'role': 'user', 'content': 'LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택'}, {'role': 'assistant', 'content': 'LG G8 씽큐 15일부터 예판…디스플레이 1회 무상교체 혜택'}, {'role': 'user', 'content': '서울문고 N풍그U에 인수… f라인서점 교m영풍y양강d도로'}, {'role': 'assistant', 'content': '서울문고, 영풍그

In [ ]:
def generate_sentence(messages):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=False,
        temperature=0.8,
        top_p=0.9,
        num_beams=4
    )
    decoded_data = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    # generate_data.append(decoded_data)
    print(decoded_data)
generate_sentence(messages=messages)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


산업생산 째 고 제조 가동률은 금융위 수 합 보


: 

In [57]:
# Noise Data load
data_path = os.path.join('..', '..', 'data',"train_Noise_data_ascii.csv")
data = pd.read_csv(data_path)
data = data.drop(columns=["Unnamed: 0"])
text_data = data["text"]

In [59]:
generate_data = []
for inp_text in text_data:
    messages = [
        [{"role":"system", "content":system_message}] +
        few_shot_messages +
        [{"role":"user", "content":inp_text}]
    ]
    generate_sentence(messages)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_m

In [63]:
data["processed_data"] = generate_data
da = data
da.to_csv("generated_data.csv", encoding="utf-8-sig")